In [ ]:
%matplotlib inline
import os

import numpy as np
import xarray as xr
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import patheffects
from matplotlib.ticker import LogLocator, NullFormatter
from scipy.stats import genpareto, lognorm

import seaborn as sns
context = 'talk'
sns.set_context(context)
#sns.set_style('whitegrid')

plt.rcParams.update({
    "figure.facecolor":  (1.0, 1.0, 1.0, 1),  # red   with alpha = 30%
    "axes.facecolor":    (1.0, 1.0, 1.0, 1),  # green with alpha = 50%
    "savefig.facecolor": (1.0, 1.0, 1.0, 1),  # blue  with alpha = 20%
})
majlocator = LogLocator()

minlocator = LogLocator(base=10.0, subs=np.arange(0.1, 1, 0.1), numticks=12)
intervals = np.array([5, 10, 25, 50, 100, 250, 500, 1000, 2000, 5000])
pe = patheffects.withStroke(linewidth=5, foreground='w')

In [ ]:
mu, xi, sigma = 0, .5, 15
npyr = 365.25
baservs = lognorm.rvs(xi, mu, sigma, size=int(1000*npyr))
cdist, cbins, patches = plt.hist(baservs, bins=np.arange(0, 100, 5), density=True)


In [ ]:
lognorm.stats(xi, mu, sigma, moments='mvsk')

In [ ]:
pct = 0.99
rate = (1. - pct)
threshold = np.percentile(baservs, pct)
y = baservs[baservs > threshold] - threshold
# Fit the tail of the distribution above the threshold
gpd = genpareto.fit(y)

def returnLevels(intervals, mu, xi, sigma, rate, npyr=365.25):
    rp = mu + (sigma / xi) * (np.power(intervals * npyr * rate, xi) - 1.)
    return rp

aep = 1./intervals
fig, ax = plt.subplots(1, 1, figsize=(10,6))
baservals = returnLevels(intervals, gpd[1], gpd[0], gpd[2], rate)
rvals2 = returnLevels(intervals, gpd[1], gpd[0], gpd[2], rate/2)
rvals2b = returnLevels(intervals, gpd[1], gpd[0], gpd[2], rate*1.5)
ax.semilogy(baservals, aep, color='#2E86C1', label='Current frequency', path_effects=[pe])
#ax.semilogy(rvals2b, aep, color='k', linestyle='--', label='Increased frequency')
#ax.semilogy(rvals2, aep, color='k', label='Reduced frequency')
ax.set_xlabel("Wind speed (m/s)")
ax.set_ylabel("AEP")
ax.yaxis.set_major_locator(majlocator)
ax.yaxis.set_minor_locator(minlocator)
ax.yaxis.set_minor_formatter(NullFormatter())
ax.grid()
ax.grid(which='minor', linestyle='--', linewidth=0.5)
ax.legend(fontsize='x-small')
axin = ax.inset_axes([0.1, 0.1, 0.3, 0.3])
sns.histplot(baservs, bins=np.arange(0, 100, 2.5), element="step", stat="density", fill=False, ax=axin, color='#2E86C1')
axin.yaxis.label.set_visible(False)
plt.savefig("current_ari.png", dpi=300, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,6))
ax.semilogy(baservals, aep, color='#2E86C1', label='Current frequency', path_effects=[pe])
ax.semilogy(rvals2b, aep, color='k', linestyle='--', label='Increased frequency', path_effects=[pe])
ax.semilogy(rvals2, aep, color='k', label='Reduced frequency')
ax.set_xlabel("Wind speed (m/s)")
ax.set_ylabel("AEP")
ax.yaxis.set_major_locator(majlocator)
ax.yaxis.set_minor_locator(minlocator)
ax.yaxis.set_minor_formatter(NullFormatter())
ax.grid()
ax.grid(which='minor', linestyle='--', linewidth=0.5)
ax.legend(fontsize='x-small')
axin = ax.inset_axes([0.1, 0.1, 0.3, 0.3])
sns.histplot(baservs, bins=np.arange(0, 100, 2.5), element="step", stat="density", fill=False, ax=axin, color='#2E86C1')
axin.yaxis.label.set_visible(False)
plt.savefig("dfreq_ari.png", dpi=300, bbox_inches="tight")

This figure shows the change in AEP for given wind speeds, when the frequency is reduced by 50%. The inset figure shows the distribution of wind speeds. In this case, the distributions are identical as they normalised (area below the curve is unity). The result of reducing the frequency is to move the AEP curve downwards. Also shown is the impact of increasing the frequency by 50%. 

In [ ]:
mu, xi, sigma = 0, .5, 17
npyr = 365.25
rvs = lognorm.rvs(xi, mu, sigma, size=int(1000*npyr))
#ax.step(cbins[:-1], cdist,  color='k')threshold = np.percentile(rvs, pct)
y = rvs[rvs>threshold] - threshold
# Fit the tail of the distribution above the threshold
gpd = genpareto.fit(y)
rvals3 = returnLevels(intervals, gpd[1], gpd[0], gpd[2], rate)

fig, ax = plt.subplots(1, 1, figsize=(10,6))

ax.semilogy(baservals, aep, color='#2E86C1', label='Current intensity', path_effects=[pe])
ax.semilogy(rvals3, aep, color='k', label='Increased mean intensity', path_effects=[pe])
ax.set_xlabel("Wind speed (m/s)")
ax.set_ylabel("AEP")
ax.yaxis.set_major_locator(majlocator)
ax.yaxis.set_minor_locator(minlocator)
ax.yaxis.set_minor_formatter(NullFormatter())
ax.grid()
ax.grid(which='minor', linestyle='--', linewidth=0.5)
ax.legend(fontsize='x-small')

axin = ax.inset_axes([0.1, 0.1, 0.3, 0.3])
sns.histplot(rvs, bins=np.arange(0, 100, 2.5), element="step", stat="density", fill=False, ax=axin, color='k')
sns.histplot(baservs, bins=np.arange(0, 100, 2.5), element="step", stat="density", fill=False, ax=axin, color='#2E86C1')
axin.yaxis.label.set_visible(False)
plt.savefig("dmean_ari.png", dpi=300, bbox_inches="tight")

In [ ]:
mu, xi, sigma = 0, .55, 15
npyr = 365.25
rvs = lognorm.rvs(xi, mu, sigma, size=int(1000*npyr))

threshold = np.percentile(rvs, pct)
y = rvs[rvs>threshold] - threshold
# Fit the tail of the distribution above the threshold
gpd = genpareto.fit(y)
rvals4 = returnLevels(intervals, gpd[1], gpd[0], gpd[2], rate)
fig, ax = plt.subplots(1, 1, figsize=(10,6))

ax.semilogy(baservals, aep, color='#2E86C1', label='Current intensity', path_effects=[pe])
ax.semilogy(rvals4, aep, color='k', label='Distribution shift', path_effects=[pe])
ax.set_xlabel("Wind speed (m/s)")
ax.set_ylabel("AEP")
ax.yaxis.set_major_locator(majlocator)
ax.yaxis.set_minor_locator(minlocator)
ax.yaxis.set_minor_formatter(NullFormatter())
ax.grid()
ax.grid(which='minor', linestyle='--', linewidth=0.5)
ax.legend(fontsize='x-small')

axin = ax.inset_axes([0.1, 0.1, 0.3, 0.3])
sns.histplot(rvs, bins=np.arange(0, 100, 2.5), element="step", stat="density", fill=False, ax=axin, color='k')
sns.histplot(baservs, bins=np.arange(0, 100, 2.5), element="step", stat="density", fill=False, ax=axin, color='#2E86C1')
axin.yaxis.label.set_visible(False)
plt.savefig("ddist_ari.png", dpi=300, bbox_inches="tight")

In [ ]:
mu, xi, sigma = 0, .55, 15
npyr = 365.25
rvs = lognorm.rvs(xi, mu, sigma, size=int(1000*npyr))

threshold = np.percentile(rvs, pct)
y = rvs[rvs>threshold] - threshold
# Fit the tail of the distribution above the threshold
gpd = genpareto.fit(y)
rvals5 = returnLevels(intervals, gpd[1], gpd[0], gpd[2], rate*.5)
fig, ax = plt.subplots(1, 1, figsize=(10,6))

ax.semilogy(baservals, aep, color='#2E86C1', label='Current intensity', path_effects=[pe])
ax.semilogy(rvals5, aep, color='k', label='Distribution shift\nreduced frequency', path_effects=[pe])
ax.set_xlabel("Wind speed (m/s)")
ax.set_ylabel("AEP")
ax.yaxis.set_major_locator(majlocator)
ax.yaxis.set_minor_locator(minlocator)
ax.yaxis.set_minor_formatter(NullFormatter())
ax.grid()
ax.grid(which='minor', linestyle='--', linewidth=0.5)
ax.legend(fontsize='x-small')
axin = ax.inset_axes([0.1, 0.1, 0.3, 0.3])
sns.histplot(rvs, bins=np.arange(0, 100, 2.5), element="step", stat="density", fill=False, ax=axin, color='k')
sns.histplot(baservs, bins=np.arange(0, 100, 2.5), element="step", stat="density", fill=False, ax=axin, color='#2E86C1')
axin.yaxis.label.set_visible(False)

x = baservals[np.argmin(np.abs(baservals - rvals5))]
y = aep[np.argmin(np.abs(baservals - rvals5))]
ax.annotate("Crossover", xy=(x+5, y*0.95), xytext=(85, 0.02),
            arrowprops=dict(arrowstyle='->', facecolor='k'))
None
fig.tight_layout()
plt.savefig("dfreqdist_ari.png", dpi=300, bbox_inches="tight")

The crossover point above will vary depending on the combination of the intensity distribution change and the frequency change. A larger decrease in the frequency would shift the crossover point further to the right (we use a 50% reduction in this example).  